In this notebook, we'll explore the basics of raster image processing in open source Python. Here, we will stack raw Landsat bands to a single 4-band Geotiff and calculate the Normalized Difference Vegetation Index (NDVI) for the Austin area, a typical metric for vegetation health.

In [1]:
import os
import numpy as np
from osgeo import gdal, ogr, osr
import matplotlib.pyplot as plt

%matplotlib inline

Since I can't store Landsat imagery on Github, download a Landsat image from the USGS. Typically they come as separate Geotiffs for each band. Here, I stored them all in one directory of raw landsat images. Specify this directory as well as an output directory.

In [2]:
#In directory with the original 8 bands:
in_tif_dir = r"C:\Users\moult\Documents\geospatial\data\austin\austin_raw_landsat\2020_04_12"
#Output directory to save the stacked image to
out_tif_dir = r"C:\Users\moult\Documents\geospatial\data\austin\austin_merged_landsat"
#Output geotiff name, with extension
out_tif_name = 'Austin_Landsat_NDVI_2020.tif'

#Checking if directory exists
if not os.path.exists(out_tif_dir):
    os.mkdir(out_tif_dir)
    
out_tif_path = os.path.join(out_tif_dir,
                            out_tif_name)

We now get to dive into GDAL head first. Here, we instantiate the Geotiff driver for reading and writing Geotiff format:

In [3]:
tif_driver = gdal.GetDriverByName('GTiff')

Next, we will walk through all the files in the in tif directory to find the appropriate paths. The band reference list variable corresponds to the Red, Blue, Green, and NIR bands respectively which we will be using for the NDVI.

For each geotiff, we use gdal to open the file and read the raster band as a numpy array. We then append that array to a list of arrayts to eventually stack.

In [4]:
band_reference_list = [2, 3, 4, 5]
tif_arr_list = []

for dirs, subdirs, files in os.walk(in_tif_dir):
    for file in files:
        fname = file.split('.')[0]
        ext = file.split('.')[-1].lower()
        if ext == 'tif':
            for band_num in band_reference_list:
                if fname.endswith('B{}'.format(band_num)):
                    geotiff_path = os.path.join(dirs, file)
                    ds = gdal.Open(geotiff_path)
                    arr = ds.GetRasterBand(1).ReadAsArray()       
                    tif_arr_list.append(arr)
in_array = np.array(tif_arr_list)

Now that we've got the 4 bands required, let's compute NDVI quickly:

In [5]:
red = in_array[0,:,:]
nir = in_array[3,:,:]

ndvi = (nir - red) / (nir + red)

C:\Users\moult\Anaconda3\envs\spatial_analysis\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


Next, we open the most recent geotiff to gather the appropriate spatial reference information. Specifically, the Geotransform (a 6-value tuple which defines the position, rotation, and pixel size) and the projection. 

In [6]:
in_ds = gdal.Open(geotiff_path)
in_gt = in_ds.GetGeoTransform()
prj = in_ds.GetProjection()
srs = osr.SpatialReference(wkt=prj)

Here, we create a new geotiff data source through the geotiff driver. We accomplish this by specifying the file path, shape in x and y, number of bands, and data type. We then specify the projection and geotransform:

In [7]:
out_tif = tif_driver.Create(out_tif_path, 
                            tif_arr_list[0].shape[1],
                            tif_arr_list[0].shape[0],
                            len(tif_arr_list)+1,
                            gdal.GDT_Float64)

out_tif.SetProjection(srs.ExportToWkt())
out_tif.SetGeoTransform(in_gt);

Lastly, we loop through each array in the original geotiff's arrays and write them to the multi-band image:

In [9]:
for i in range(1, len(tif_arr_list)+1):
    out_band = out_tif.GetRasterBand(i)
    out_band.WriteArray(arr)
    
ndvi_band = out_tif.GetRasterBand(5)
ndvi_band.WriteArray(ndvi)
out_tif.FlushCache()
del out_tif

Voila! In just a handful of lines of code we just stacked 4 separate geotiffs into a single multiband geotiff. Let's take a look at the results:

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

in_array = np.dstack(np.array(tif_arr_list))

ax.imshow(in_array[:,:,3], cmap=plt.cm.Greys_r)

How about we compute NDVI in just a couple lines?

In [ ]:
out_ndvi = tif_driver.Create(out_tif_path, 
                            tif_arr_list[0].shape[1],
                            tif_arr_list[0].shape[0],
                            len(tif_arr_list),
                            gdal.GDT_Float64)

out_tif.SetProjection(srs.ExportToWkt())
out_tif.SetGeoTransform(in_gt);